In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
import numpy as np
import os

In [9]:
from IPython import get_ipython
pwd= os.path.abspath(get_ipython().starting_dir)
root_dir = os.path.join(pwd,"..")
dataset_path = os.path.join(root_dir,"Dataset","titanic_dataset.csv")


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [11]:
df = pd.read_csv(dataset_path)
df.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
735,736,0,3,"Williams, Mr. Leslie",male,28.5,0,0,54636,16.100,NaN,S
350,351,0,3,"Odahl, Mr. Nils Martin",male,23.0,0,0,7267,9.225,NaN,S
374,375,0,3,"Palsson, Miss. Stina Viola",female,3.0,3,1,349909,21.075,NaN,S
144,145,0,2,"Andrew, Mr. Edgardo Samuel",male,18.0,0,0,231945,11.500,NaN,S
719,720,0,3,"Johnson, Mr. Malkolm Joackim",male,33.0,0,0,347062,7.775,NaN,S


In [87]:
#feature reduction
df = df.drop(columns=["PassengerId","Name","Ticket","Cabin"])

In [88]:
from sklearn.model_selection import train_test_split
x_test,x_train,y_test,y_train = train_test_split(df.drop(columns=["Survived"]),df["Survived"],random_state=42,train_size=0.20)
x_test

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
761,3,male,41.0,0,0,7.1250,S
645,1,male,48.0,1,0,76.7292,C
754,2,female,48.0,1,2,65.0000,S
556,1,female,48.0,1,0,39.6000,C
850,3,male,4.0,4,2,31.2750,S
...,...,...,...,...,...,...,...
106,3,female,21.0,0,0,7.6500,S
270,1,male,NaN,0,0,31.0000,S
860,3,male,41.0,2,0,14.1083,S
435,1,female,14.0,1,2,120.0000,S


In [89]:
#filling the missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [90]:
df["Sex"] = df["Sex"].astype("category") #memory optimisation
x_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
709,3,male,NaN,1,1,15.2458,C
439,2,male,31.00,0,0,10.5000,S
840,3,male,20.00,0,0,7.9250,S
720,2,female,6.00,0,1,33.0000,S
39,3,female,14.00,1,0,11.2417,C
...,...,...,...,...,...,...,...
639,3,male,NaN,1,0,16.1000,S
878,3,male,NaN,0,0,7.8958,S
824,3,male,2.00,4,1,39.6875,S
803,3,male,0.42,0,1,8.5167,C


In [91]:
si_age = SimpleImputer() # filling mean value of age
si_embarked = SimpleImputer(strategy="most_frequent") # filling mode of embarked value

In [92]:
x_train_age = si_age.fit_transform(x_train[["Age"]])
x_train_embarked = si_embarked.fit_transform(x_train[["Embarked"]]) #i dont know the reason why double [[]] is used

In [93]:
x_test_age = si_age.transform(x_test[["Age"]])
x_test_embarked= si_embarked.transform(x_test[["Embarked"]])

In [94]:
x_train_age = x_train_age.astype("int")
x_test_age = x_test_age.astype("int")
x_train.drop(columns=["Age","Embarked"],inplace=True)
x_test.drop(columns=["Age","Embarked"],inplace=True)

In [95]:
x_train = pd.DataFrame(np.concatenate((np.array(x_train),np.array(x_train_age),np.array(x_train_embarked)),axis=1))
x_train

,0,1,2,3,4,5,6
0,3,male,1,1,15.2458,29,C
1,2,male,0,0,10.5,31,S
2,3,male,0,0,7.925,20,S
3,2,female,0,1,33.0,6,S
4,3,female,1,0,11.2417,14,C
...,...,...,...,...,...,...,...
708,3,male,1,0,16.1,29,S
709,3,male,0,0,7.8958,29,S
710,3,male,4,1,39.6875,2,S
711,3,male,0,1,8.5167,0,C


In [96]:
x_train.set_index(y_train.index,inplace=True)


In [97]:
x_test = pd.DataFrame(np.concatenate((np.array(x_test),np.array(x_test_age),np.array(x_test_embarked)),axis=1))
x_test.set_index(y_test.index,inplace=True)
x_test

,0,1,2,3,4,5,6
761,3,male,0,0,7.125,41,S
645,1,male,1,0,76.7292,48,C
754,2,female,1,2,65.0,48,S
556,1,female,1,0,39.6,48,C
850,3,male,4,2,31.275,4,S
...,...,...,...,...,...,...,...
106,3,female,0,0,7.65,21,S
270,1,male,0,0,31.0,29,S
860,3,male,2,0,14.1083,41,S
435,1,female,1,2,120.0,14,S


In [98]:
x_train[1]

709      male
439      male
840      male
720    female
39     female
        ...  
639      male
878      male
824      male
803      male
619      male
Name: 1, Length: 713, dtype: object

In [99]:
#applying one hot encoding
ohe_sex = OneHotEncoder(handle_unknown="ignore",sparse_output=False)
ohe_embarked = OneHotEncoder(handle_unknown="ignore",sparse_output=False)


In [100]:
sex_train = ohe_sex.fit_transform(x_train[[1]])
sex_test = ohe_sex.transform(x_test[[1]]) # sex column
sex_train

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]])

In [101]:
Embarked_train = ohe_embarked.fit_transform(x_train[[6]])
Embarked_test = ohe_embarked.transform(x_test[[6]])# embarked columns
Embarked_train

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.]])

In [102]:
#in decision tree we dont have to remove one columns as there is no case of multiple collinearity
x_train.drop(columns=[1,6],inplace=True)
x_test.drop(columns=[1,6],inplace=True)

In [103]:
x_train

,0,2,3,4,5
709,3,1,1,15.2458,29
439,2,0,0,10.5,31
840,3,0,0,7.925,20
720,2,0,1,33.0,6
39,3,1,0,11.2417,14
...,...,...,...,...,...
639,3,1,0,16.1,29
878,3,0,0,7.8958,29
824,3,4,1,39.6875,2
803,3,0,1,8.5167,0


In [104]:
x_train_transformed = np.concatenate((x_train,sex_train,Embarked_train),axis=1)
x_test_transformed = np.concatenate((x_test,sex_test,Embarked_test),axis=1)

In [105]:
pd.value_counts(df["Embarked"])

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [106]:
x_train = pd.DataFrame(x_train_transformed,columns=["Pclass","SibSp","Parch","Fare","Age","Female","Male","C","Q","S"]).set_index(y_train.index)
x_test = pd.DataFrame(x_test_transformed,columns=["Pclass","SibSp","Parch","Fare","Age","Female","Male","C","Q","S"]).set_index(y_test.index)
x_train,y_train

(    Pclass SibSp Parch     Fare Age Female Male    C    Q    S
 709      3     1     1  15.2458  29    0.0  1.0  1.0  0.0  0.0
 439      2     0     0     10.5  31    0.0  1.0  0.0  0.0  1.0
 840      3     0     0    7.925  20    0.0  1.0  0.0  0.0  1.0
 720      2     0     1     33.0   6    1.0  0.0  0.0  0.0  1.0
 39       3     1     0  11.2417  14    1.0  0.0  1.0  0.0  0.0
 ..     ...   ...   ...      ...  ..    ...  ...  ...  ...  ...
 639      3     1     0     16.1  29    0.0  1.0  0.0  0.0  1.0
 878      3     0     0   7.8958  29    0.0  1.0  0.0  0.0  1.0
 824      3     4     1  39.6875   2    0.0  1.0  0.0  0.0  1.0
 803      3     0     1   8.5167   0    0.0  1.0  1.0  0.0  0.0
 619      2     0     0     10.5  26    0.0  1.0  0.0  0.0  1.0
 
 [713 rows x 10 columns],
 709    1
 439    0
 840    0
 720    1
 39     1
       ..
 639    0
 878    0
 824    0
 803    1
 619    0
 Name: Survived, Length: 713, dtype: int64)

In [107]:
#Model_bulding
dt = DecisionTreeClassifier()

In [108]:
dt.fit(x_train,y_train)

DecisionTreeClassifier()

In [109]:
y_predict=dt.predict(x_test)

In [110]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_predict)

0.797752808988764

In [111]:
#exporting the model
#pickle serialize the data and than saves the file in specified location
import pickle


In [112]:
pickle.dump(ohe_sex,open(r'E:\New folder (2)\pipeline\pickled_file\ohe_sex.pkl', 'wb'))
#if file doesn't exist than it will create and if there it will be over written

In [113]:
pickle.dump(ohe_embarked,open(r"E:\New folder (2)\pipeline\pickled_file\ohe_embarked.pkl","wb"))

In [114]:
pickle.dump(dt,open(r"E:\New folder (2)\pipeline\pickled_file\dt.pkl","wb"))